In [441]:
import datetime
from datetime import datetime, timedelta
import os.path
import os

today = datetime.now()

###########################Getting Dates for last week################################


print(today)
def dow(date):
    days=["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
    dayNumber=date.weekday()
    print(dayNumber)
    return dayNumber
    
def getSaturday():
    today = datetime.now()
    date = today.weekday()
    if date == 0:
        d = today - timedelta(days=2)
        return d
    elif date == 1:
        d = today - timedelta(days=3)
        #print(type(d))
        return d
    elif date == 2:
        d = today - timedelta(days=4)
        return d
    elif date == 3:
        d = today - timedelta(days=5)
        return d
    elif date == 4:
        d = today - timedelta(days=6)
        return d
    elif date == 5:
        d = today - timedelta(days=7)
        return d
    else:
        d = today - timedelta(days=1)
        return d
        

dateA = getSaturday()-timedelta(days=6)
strDateA = str(dateA)
strSaturday = str(getSaturday())

year, month, dateRest = map(str, strDateA.split('-'))
day, time = map(str, dateRest.split(' '))

print(year)
print(month)
print(day)
print(time)

2021-08-08 03:06:00.744438
2021
08
01
03:06:00.746432


In [442]:
from datetime import *
today = getSaturday()
print (today.strftime("%U"))
print(getSaturday())

31
2021-08-07 03:06:00.765382


In [443]:

FHD_days = ['Sunday', 'Monday', 'Tuesday']

BHD_days = ['Thursday', 'Friday', 'Saturday']

FHD_start = []
FHD_end = []

BHD_start = []
BHD_end = []

FHN_start = []
FHN_end = []

BHN_start = []
BHN_end = []

for x in range(6,3,-1):
    dateA = getSaturday() - timedelta(days=x)
    FHD_start.append(dateA)
    FHN_start.append(dateA)
    FHD_end.append(dateA)
    
for x in range(5,2,-1):
    dateA = getSaturday() - timedelta(days=x)
    FHN_end.append(dateA)
  
    
for x in range(2,-1,-1):
    dateA = getSaturday() - timedelta(days=x)
    BHD_start.append(dateA)
    BHD_end.append(dateA)
    BHN_start.append(dateA)
    
for x in range(1,-2,-1):
    dateA = getSaturday() - timedelta(days=x)
    BHN_end.append(dateA)
    
BHN_start

BHN_end



[datetime.datetime(2021, 8, 6, 3, 6, 0, 799295),
 datetime.datetime(2021, 8, 7, 3, 6, 0, 799295),
 datetime.datetime(2021, 8, 8, 3, 6, 0, 799295)]

In [444]:
#Function Roll up Worksheet
#Nike IDS Stow rates
#StowPPR = f'https://fclm-portal.amazon.com/reports/functionRollup?reportFormat=HTML&warehouseId=SIL2&processId=1002976&maxIntradayDays=1&spanType=Intraday&startDateIntraday={startYear}%2F{startMonth}%2F{startDay}&startHourIntraday={startHr}&startMinuteIntraday={startMin}&endDateIntraday={endYear}%2F{endMonth}%2F{endDay}&endHourIntraday={endHr}&endMinuteIntraday={endMin}'


#Used for pulling Chute rates
#ChuteRatesPPR = f'https://fclm-portal.amazon.com/reports/functionRollup?reportFormat=CSV&warehouseId=SIL2&processId=1002969&maxIntradayDays=1&spanType=Intraday&startDateIntraday={startYear}%2F{startMonth}%2F{startDay}&startHourIntraday={startHr}&startMinuteIntraday={startMin}&endDateIntraday={endYear}%2F{endMonth}%2F{endDay}&endHourIntraday={endHr}&endMinuteIntraday={endMinute}'


#Used for pulling pick rates(PPR)
PPRPickrates = f'https://fclm-portal.amazon.com/reports/functionRollup?reportFormat=HTML&warehouseId=SIL2&processId=1003001&maxIntradayDays=1&spanType=Intraday&startDateIntraday=2021%2F07%2F09&startHourIntraday=21&startMinuteIntraday=0&endDateIntraday=2021%2F07%2F10&endHourIntraday=6&endMinuteIntraday=30'

functionRollUp = [StowPPR, ChuteRatesPPR, PPRPickrates]

In [445]:
import argparse
import io
import json
import os
import sys
from typing import Final, List

import pandas as pd
import pendulum
import requests
from bs4 import BeautifulSoup
#from pandas import DataFrame
import urllib3
from urllib3.exceptions import InsecureRequestWarning
from requests_kerberos import OPTIONAL, HTTPKerberosAuth
urllib3.disable_warnings()
    
def funcRollChuteFHD():
    
    if os.path.exists('ChuteRatesPPR_FHD.csv')== False:
        df_empty = pd.DataFrame()
        for x in range(3):
            strDate = str(FHD_start[x])
            strDate2 = str(FHD_end[x])
            startYear, startMonth, dateRest = map(str, strDate.split('-'))
            startDay, time = map(str, dateRest.split(' '))

            endYear, endMonth, dateRest2 = map(str, strDate2.split('-'))
            endDay, time = map(str, dateRest2.split(' '))

            startHr = '06'
            startMin ='30'

            endHr = '20'
            endMin = '00'


            URL = f'https://fclm-portal.amazon.com/reports/functionRollup?reportFormat=CSV&warehouseId=SIL2&processId=1002969&maxIntradayDays=1&spanType=Intraday&startDateIntraday={startYear}%2F{startMonth}%2F{startDay}&startHourIntraday={startHr}&startMinuteIntraday={startMin}&endDateIntraday={endYear}%2F{endMonth}%2F{endDay}&endHourIntraday={endHr}&endMinuteIntraday={endMin}'
            req = requests.Session()
            resp = req.get(URL, verify=False, allow_redirects=True, auth=HTTPKerberosAuth(mutual_authentication=OPTIONAL))
            data = io.StringIO(resp.text)
            df = pd.read_csv(data, usecols=['Employee Id', 'Paid Hours-Total(function,employee)', 'Jobs'])

            df.drop_duplicates(subset=['Employee Id'], keep='last', inplace=True)
            #help(df.drop)
            df = df.drop(columns=['Employee Id'])
            df = df.rename(columns = {'Paid Hours-Total(function,employee)': 'Total Hours'})

            #df.info()

            total = FHD_days[x]+" "+ "FHD "+ str(startMonth)+"/"+str(startDay)
            df.loc[total] = df.sum()
            df = df.astype({"Total Hours": float, "Jobs": int})
            df.round({'Total Hours': 2})
            totalRow=df.tail(1)
            df1 = pd.DataFrame(totalRow)
            #print(df1)

            df_empty = df_empty.append(df1)
        df_empty.to_csv("ChuteRatesPPR_FHD.csv")
        print(df_empty)
    else:
        print('Chute Rates for FHDs already collected! Continuing to next grab')
funcRollChuteFHD()

Chute Rates for FHDs already collected! Continuing to next grab


In [446]:
def funcRollChuteBHD():
    if os.path.exists('ChuteRatesPPR_BHD.csv')== False:
        df_empty = pd.DataFrame()
        for x in range(3):
            strDate = str(BHD_start[x])
            strDate2 = str(BHD_end[x])
            startYear, startMonth, dateRest = map(str, strDate.split('-'))
            startDay, time = map(str, dateRest.split(' '))

            endYear, endMonth, dateRest2 = map(str, strDate2.split('-'))
            endDay, time = map(str, dateRest2.split(' '))

            startHr = '06'
            startMin ='30'

            endHr = '20'
            endMin = '00'


            URL = f'https://fclm-portal.amazon.com/reports/functionRollup?reportFormat=CSV&warehouseId=SIL2&processId=1002969&maxIntradayDays=1&spanType=Intraday&startDateIntraday={startYear}%2F{startMonth}%2F{startDay}&startHourIntraday={startHr}&startMinuteIntraday={startMin}&endDateIntraday={endYear}%2F{endMonth}%2F{endDay}&endHourIntraday={endHr}&endMinuteIntraday={endMin}'
            req = requests.Session()
            resp = req.get(URL, verify=False, allow_redirects=True, auth=HTTPKerberosAuth(mutual_authentication=OPTIONAL))
            data = io.StringIO(resp.text)
            df = pd.read_csv(data, usecols=['Employee Id', 'Paid Hours-Total(function,employee)', 'Jobs'])

            df.drop_duplicates(subset=['Employee Id'], keep='last', inplace=True)
            #help(df.drop)
            df = df.drop(columns=['Employee Id'])
            df = df.rename(columns = {'Paid Hours-Total(function,employee)': 'Total Hours'})

            #df.info()

            total = BHD_days[x]+" "+ "BHD "+ str(startMonth)+"/"+str(startDay)
            df.loc[total] = df.sum()
            df = df.astype({"Total Hours": float, "Jobs": int})
            df.round({'Total Hours': 2})
            totalRow=df.tail(1)
            df1 = pd.DataFrame(totalRow)
            #print(df1)

            df_empty = df_empty.append(df1)
        df_empty.to_csv("ChuteRatesPPR_BHD.csv")
        print(df_empty)
    else:
        print('Chute Rates for BHDs already collected! Continuing to next grab')
funcRollChuteBHD()

Chute Rates for BHDs already collected! Continuing to next grab


In [447]:
def funcRollChuteFHN():
    if os.path.exists('ChuteRatesPPR_FHN.csv')== False:
        df_empty = pd.DataFrame()
        for x in range(3):
            strDate = str(FHN_start[x])
            strDate2 = str(FHN_end[x])
            startYear, startMonth, dateRest = map(str, strDate.split('-'))
            startDay, time = map(str, dateRest.split(' '))

            endYear, endMonth, dateRest2 = map(str, strDate2.split('-'))
            endDay, time = map(str, dateRest2.split(' '))

            startHr = '20'
            startMin ='00'

            endHr = '06'
            endMin = '30'


            URL = f'https://fclm-portal.amazon.com/reports/functionRollup?reportFormat=CSV&warehouseId=SIL2&processId=1002969&maxIntradayDays=1&spanType=Intraday&startDateIntraday={startYear}%2F{startMonth}%2F{startDay}&startHourIntraday={startHr}&startMinuteIntraday={startMin}&endDateIntraday={endYear}%2F{endMonth}%2F{endDay}&endHourIntraday={endHr}&endMinuteIntraday={endMin}'
            req = requests.Session()
            resp = req.get(URL, verify=False, allow_redirects=True, auth=HTTPKerberosAuth(mutual_authentication=OPTIONAL))
            data = io.StringIO(resp.text)
            df = pd.read_csv(data, usecols=['Employee Id', 'Paid Hours-Total(function,employee)', 'Jobs'])

            df.drop_duplicates(subset=['Employee Id'], keep='last', inplace=True)
            #help(df.drop)
            df = df.drop(columns=['Employee Id'])
            df = df.rename(columns = {'Paid Hours-Total(function,employee)': 'Total Hours'})

            #df.info()

            total = FHD_days[x]+" "+ "FHN "+ str(startMonth)+"/"+str(startDay)
            df.loc[total] = df.sum()
            df = df.astype({"Total Hours": float, "Jobs": int})
            df.round({'Total Hours': 2})
            totalRow=df.tail(1)
            df1 = pd.DataFrame(totalRow)
            #print(df1)

            df_empty = df_empty.append(df1)
        df_empty.to_csv("ChuteRatesPPR_FHN.csv")
        print(df_empty)
    else:
        print('Chute Rates for FHNs already collected! Continuing to next grab')
funcRollChuteFHN()

Chute Rates for FHNs already collected! Continuing to next grab


In [448]:
def funcRollChuteBHN():
    if os.path.exists('ChuteRatesPPR_BHN.csv')== False:
        df_empty = pd.DataFrame()
        for x in range(3):
            strDate = str(BHN_start[x])
            strDate2 = str(BHN_end[x])
            startYear, startMonth, dateRest = map(str, strDate.split('-'))
            startDay, time = map(str, dateRest.split(' '))

            endYear, endMonth, dateRest2 = map(str, strDate2.split('-'))
            endDay, time = map(str, dateRest2.split(' '))

            startHr = '20'
            startMin ='00'

            endHr = '06'
            endMin = '30'


            URL = f'https://fclm-portal.amazon.com/reports/functionRollup?reportFormat=CSV&warehouseId=SIL2&processId=1002969&maxIntradayDays=1&spanType=Intraday&startDateIntraday={startYear}%2F{startMonth}%2F{startDay}&startHourIntraday={startHr}&startMinuteIntraday={startMin}&endDateIntraday={endYear}%2F{endMonth}%2F{endDay}&endHourIntraday={endHr}&endMinuteIntraday={endMin}'
            req = requests.Session()
            resp = req.get(URL, verify=False, allow_redirects=True, auth=HTTPKerberosAuth(mutual_authentication=OPTIONAL))
            data = io.StringIO(resp.text)
            df = pd.read_csv(data, usecols=['Employee Id', 'Paid Hours-Total(function,employee)', 'Jobs'])

            df.drop_duplicates(subset=['Employee Id'], keep='last', inplace=True)
            #help(df.drop)
            df = df.drop(columns=['Employee Id'])
            df = df.rename(columns = {'Paid Hours-Total(function,employee)': 'Total Hours'})

            #df.info()

            total = BHD_days[x]+" "+ "BHN "+ str(startMonth)+"/"+str(startDay)
            df.loc[total] = df.sum()
            df = df.astype({"Total Hours": float, "Jobs": int})
            df.round({'Total Hours': 2})
            totalRow=df.tail(1)
            df1 = pd.DataFrame(totalRow)
            #print(df1)

            df_empty = df_empty.append(df1)
        df_empty.to_csv("ChuteRatesPPR_BHN.csv")
        print(df_empty)
    else:
        print('Chute Rates for BHNs already collected! Continuing to next grab')
    
funcRollChuteBHN()
print(datetime.now())

Chute Rates for BHNs already collected! Continuing to next grab
2021-08-08 03:06:01.023709


In [449]:
#StowPPR = f'https://fclm-portal.amazon.com/reports/functionRollup?reportFormat=CSV&warehouseId=SIL2&processId=1002976&maxIntradayDays=1&spanType=Intraday&startDateIntraday={startYear}%2F{startMonth}%2F{startDay}&startHourIntraday={startHr}&startMinuteIntraday={startMin}&endDateIntraday={endYear}%2F{endMonth}%2F{endDay}&endHourIntraday={endHr}&endMinuteIntraday={endMin}'

def funcRollNikeStowFHD():
    
    if os.path.exists('NikeStow_FHD.csv')== False:
        df_empty = pd.DataFrame()
        for x in range(3):
            strDate = str(FHD_start[x])
            strDate2 = str(FHD_end[x])
            startYear, startMonth, dateRest = map(str, strDate.split('-'))
            startDay, time = map(str, dateRest.split(' '))

            endYear, endMonth, dateRest2 = map(str, strDate2.split('-'))
            endDay, time = map(str, dateRest2.split(' '))

            startHr = '06'
            startMin ='30'

            endHr = '20'
            endMin = '00'


            URL = f'https://fclm-portal.amazon.com/reports/functionRollup?reportFormat=CSV&warehouseId=SIL2&processId=1002976&maxIntradayDays=1&spanType=Intraday&startDateIntraday={startYear}%2F{startMonth}%2F{startDay}&startHourIntraday={startHr}&startMinuteIntraday={startMin}&endDateIntraday={endYear}%2F{endMonth}%2F{endDay}&endHourIntraday={endHr}&endMinuteIntraday={endMin}'
            req = requests.Session()
            resp = req.get(URL, verify=False, allow_redirects=True, auth=HTTPKerberosAuth(mutual_authentication=OPTIONAL))
            data = io.StringIO(resp.text)
            df = pd.read_csv(data, usecols=['Employee Id', 'Paid Hours-Total(function,employee)', 'Jobs'])

            df.drop_duplicates(subset=['Employee Id'], keep='last', inplace=True)
            #help(df.drop)
            df = df.drop(columns=['Employee Id'])
            df = df.rename(columns = {'Paid Hours-Total(function,employee)': 'Total Hours'})

            #df.info()

            total = FHD_days[x]+" "+ "FHD "+ str(startMonth)+"/"+str(startDay)
            df.loc[total] = df.sum()
            df = df.astype({"Total Hours": float, "Jobs": int})
            df.round({'Total Hours': 2})
            totalRow=df.tail(1)
            df1 = pd.DataFrame(totalRow)
            #print(df1)

            df_empty = df_empty.append(df1)
        df_empty.to_csv("NikeStow_FHD.csv")
        print(df_empty)
    else:
        print('NikeStow for FHDs already collected! Continuing to next grab')
funcRollNikeStowFHD()

NikeStow for FHDs already collected! Continuing to next grab


In [450]:
def funcRollNikeStowBHD():
    if os.path.exists('NikeStow_BHD.csv')== False:
        df_empty = pd.DataFrame()
        for x in range(3):
            strDate = str(BHD_start[x])
            strDate2 = str(BHD_end[x])
            startYear, startMonth, dateRest = map(str, strDate.split('-'))
            startDay, time = map(str, dateRest.split(' '))

            endYear, endMonth, dateRest2 = map(str, strDate2.split('-'))
            endDay, time = map(str, dateRest2.split(' '))

            startHr = '06'
            startMin ='30'

            endHr = '20'
            endMin = '00'


            URL = f'https://fclm-portal.amazon.com/reports/functionRollup?reportFormat=CSV&warehouseId=SIL2&processId=1002976&maxIntradayDays=1&spanType=Intraday&startDateIntraday={startYear}%2F{startMonth}%2F{startDay}&startHourIntraday={startHr}&startMinuteIntraday={startMin}&endDateIntraday={endYear}%2F{endMonth}%2F{endDay}&endHourIntraday={endHr}&endMinuteIntraday={endMin}'
            req = requests.Session()
            resp = req.get(URL, verify=False, allow_redirects=True, auth=HTTPKerberosAuth(mutual_authentication=OPTIONAL))
            data = io.StringIO(resp.text)
            df = pd.read_csv(data, usecols=['Employee Id', 'Paid Hours-Total(function,employee)', 'Jobs'])

            df.drop_duplicates(subset=['Employee Id'], keep='last', inplace=True)
            #help(df.drop)
            df = df.drop(columns=['Employee Id'])
            df = df.rename(columns = {'Paid Hours-Total(function,employee)': 'Total Hours'})

            #df.info()

            total = BHD_days[x]+" "+ "BHD "+ str(startMonth)+"/"+str(startDay)
            df.loc[total] = df.sum()
            df = df.astype({"Total Hours": float, "Jobs": int})
            df.round({'Total Hours': 2})
            totalRow=df.tail(1)
            df1 = pd.DataFrame(totalRow)
            #print(df1)

            df_empty = df_empty.append(df1)
        df_empty.to_csv("NikeStow_BHD.csv")
        print(df_empty)
    else:
        print('NikeStow for BHDs already collected! Continuing to next grab...')
funcRollNikeStowBHD()

NikeStow for BHDs already collected! Continuing to next grab...


In [451]:
def funcRollNikeStowFHN():
    if os.path.exists('NikeStow_FHN.csv')== False:
        df_empty = pd.DataFrame()
        for x in range(3):
            strDate = str(FHN_start[x])
            strDate2 = str(FHN_end[x])
            startYear, startMonth, dateRest = map(str, strDate.split('-'))
            startDay, time = map(str, dateRest.split(' '))

            endYear, endMonth, dateRest2 = map(str, strDate2.split('-'))
            endDay, time = map(str, dateRest2.split(' '))

            startHr = '20'
            startMin ='00'

            endHr = '06'
            endMin = '30'


            URL = f'https://fclm-portal.amazon.com/reports/functionRollup?reportFormat=CSV&warehouseId=SIL2&processId=1002976&maxIntradayDays=1&spanType=Intraday&startDateIntraday={startYear}%2F{startMonth}%2F{startDay}&startHourIntraday={startHr}&startMinuteIntraday={startMin}&endDateIntraday={endYear}%2F{endMonth}%2F{endDay}&endHourIntraday={endHr}&endMinuteIntraday={endMin}'
            req = requests.Session()
            resp = req.get(URL, verify=False, allow_redirects=True, auth=HTTPKerberosAuth(mutual_authentication=OPTIONAL))
            data = io.StringIO(resp.text)
            df = pd.read_csv(data, usecols=['Employee Id', 'Paid Hours-Total(function,employee)', 'Jobs'])

            df.drop_duplicates(subset=['Employee Id'], keep='last', inplace=True)
            #help(df.drop)
            df = df.drop(columns=['Employee Id'])
            df = df.rename(columns = {'Paid Hours-Total(function,employee)': 'Total Hours'})

            #df.info()

            total = FHD_days[x]+" "+ "FHN "+ str(startMonth)+"/"+str(startDay)
            df.loc[total] = df.sum()
            df = df.astype({"Total Hours": float, "Jobs": int})
            df.round({'Total Hours': 2})
            totalRow=df.tail(1)
            df1 = pd.DataFrame(totalRow)
            #print(df1)

            df_empty = df_empty.append(df1)
        df_empty.to_csv("NikeStow_FHN.csv")
        print(df_empty)
    else:
        print('NikeStow for FHNs already collected! Continuing to next grab...')
funcRollNikeStowFHN()

NikeStow for FHNs already collected! Continuing to next grab...


In [452]:
def funcRollNikeStowBHN():
    if os.path.exists('NikeStow_BHN.csv')== False:
        df_empty = pd.DataFrame()
        for x in range(3):
            strDate = str(BHN_start[x])
            strDate2 = str(BHN_end[x])
            startYear, startMonth, dateRest = map(str, strDate.split('-'))
            startDay, time = map(str, dateRest.split(' '))

            endYear, endMonth, dateRest2 = map(str, strDate2.split('-'))
            endDay, time = map(str, dateRest2.split(' '))

            startHr = '20'
            startMin ='00'

            endHr = '06'
            endMin = '30'


            URL = f'https://fclm-portal.amazon.com/reports/functionRollup?reportFormat=CSV&warehouseId=SIL2&processId=1002976&maxIntradayDays=1&spanType=Intraday&startDateIntraday={startYear}%2F{startMonth}%2F{startDay}&startHourIntraday={startHr}&startMinuteIntraday={startMin}&endDateIntraday={endYear}%2F{endMonth}%2F{endDay}&endHourIntraday={endHr}&endMinuteIntraday={endMin}'
            req = requests.Session()
            resp = req.get(URL, verify=False, allow_redirects=True, auth=HTTPKerberosAuth(mutual_authentication=OPTIONAL))
            data = io.StringIO(resp.text)
            df = pd.read_csv(data, usecols=['Employee Id', 'Paid Hours-Total(function,employee)', 'Jobs'])

            df.drop_duplicates(subset=['Employee Id'], keep='last', inplace=True)
            #help(df.drop)
            df = df.drop(columns=['Employee Id'])
            df = df.rename(columns = {'Paid Hours-Total(function,employee)': 'Total Hours'})

            #df.info()

            total = BHD_days[x]+" "+ "BHN "+ str(startMonth)+"/"+str(startDay)
            df.loc[total] = df.sum()
            df = df.astype({"Total Hours": float, "Jobs": int})
            df.round({'Total Hours': 2})
            totalRow=df.tail(1)
            df1 = pd.DataFrame(totalRow)
            #print(df1)

            df_empty = df_empty.append(df1)
        df_empty.to_csv("NikeStow_BHN.csv")
        print(df_empty)
    else:
        print('NikeStow for BHNs already collected! Continuing to next grab...')
funcRollNikeStowBHN()

NikeStow for BHNs already collected! Continuing to next grab...


In [453]:
def funcRollPPPicksFHD():
    
    if os.path.exists('PPPicks_FHD.csv')== False:
        df_empty = pd.DataFrame()
        for x in range(3):
            strDate = str(FHD_start[x])
            strDate2 = str(FHD_end[x])
            startYear, startMonth, dateRest = map(str, strDate.split('-'))
            startDay, time = map(str, dateRest.split(' '))

            endYear, endMonth, dateRest2 = map(str, strDate2.split('-'))
            endDay, time = map(str, dateRest2.split(' '))

            startHr = '06'
            startMin ='30'

            endHr = '20'
            endMin = '00'


            URL = f'https://fclm-portal.amazon.com/reports/functionRollup?reportFormat=CSV&warehouseId=SIL2&processId=1003001&maxIntradayDays=1&spanType=Intraday&startDateIntraday={startYear}%2F{startMonth}%2F{startDay}&startHourIntraday={startHr}&startMinuteIntraday={startMin}&endDateIntraday={endYear}%2F{endMonth}%2F{endDay}&endHourIntraday={endHr}&endMinuteIntraday={endMin}'
            req = requests.Session()
            resp = req.get(URL, verify=False, allow_redirects=True, auth=HTTPKerberosAuth(mutual_authentication=OPTIONAL))
            data = io.StringIO(resp.text)
            df = pd.read_csv(data, usecols=['Employee Id', 'Paid Hours-Total(function,employee)', 'Jobs'])

            df.drop_duplicates(subset=['Employee Id'], keep='last', inplace=True)
            #help(df.drop)
            df = df.drop(columns=['Employee Id'])
            df = df.rename(columns = {'Paid Hours-Total(function,employee)': 'Total Hours'})

            #df.info()

            total = FHD_days[x]+" "+ "FHD "+ str(startMonth)+"/"+str(startDay)
            df.loc[total] = df.sum()
            df = df.astype({"Total Hours": float, "Jobs": int})
            df.round({'Total Hours': 2})
            totalRow=df.tail(1)
            df1 = pd.DataFrame(totalRow)
            #print(df1)

            df_empty = df_empty.append(df1)
        df_empty.to_csv("PPPicks_FHD.csv")
        print(df_empty)
    else:
        print('PPPicks for FHDs already collected! Continuing to next grab')
funcRollPPPicksFHD()

PPPicks for FHDs already collected! Continuing to next grab


In [454]:
def funcRollPPPicksBHD():
    if os.path.exists('PPPicks_BHD.csv')== False:
        df_empty = pd.DataFrame()
        for x in range(3):
            strDate = str(BHD_start[x])
            strDate2 = str(BHD_end[x])
            startYear, startMonth, dateRest = map(str, strDate.split('-'))
            startDay, time = map(str, dateRest.split(' '))

            endYear, endMonth, dateRest2 = map(str, strDate2.split('-'))
            endDay, time = map(str, dateRest2.split(' '))

            startHr = '06'
            startMin ='30'

            endHr = '20'
            endMin = '00'


            URL = f'https://fclm-portal.amazon.com/reports/functionRollup?reportFormat=CSV&warehouseId=SIL2&processId=1003001&maxIntradayDays=1&spanType=Intraday&startDateIntraday={startYear}%2F{startMonth}%2F{startDay}&startHourIntraday={startHr}&startMinuteIntraday={startMin}&endDateIntraday={endYear}%2F{endMonth}%2F{endDay}&endHourIntraday={endHr}&endMinuteIntraday={endMin}'
            req = requests.Session()
            resp = req.get(URL, verify=False, allow_redirects=True, auth=HTTPKerberosAuth(mutual_authentication=OPTIONAL))
            data = io.StringIO(resp.text)
            df = pd.read_csv(data, usecols=['Employee Id', 'Paid Hours-Total(function,employee)', 'Jobs'])

            df.drop_duplicates(subset=['Employee Id'], keep='last', inplace=True)
            #help(df.drop)
            df = df.drop(columns=['Employee Id'])
            df = df.rename(columns = {'Paid Hours-Total(function,employee)': 'Total Hours'})

            #df.info()

            total = BHD_days[x]+" "+ "BHD "+ str(startMonth)+"/"+str(startDay)
            df.loc[total] = df.sum()
            #df = df.astype({"Total Hours": float, "Jobs": int})
            df.round({'Total Hours': 2})
            totalRow=df.tail(1)
            df1 = pd.DataFrame(totalRow)
            #print(df1)

            df_empty = df_empty.append(df1)
        df_empty.to_csv("PPPicks_BHD.csv")
        print(df_empty)
    else:
        print('PPPicks for BHDs already collected! Continuing to next grab...')
funcRollPPPicksBHD()

PPPicks for BHDs already collected! Continuing to next grab...


In [455]:
def funcRollPPPicksFHN():
    if os.path.exists('PPPicks_FHN.csv')== False:
        df_empty = pd.DataFrame()
        for x in range(3):
            strDate = str(FHN_start[x])
            strDate2 = str(FHN_end[x])
            startYear, startMonth, dateRest = map(str, strDate.split('-'))
            startDay, time = map(str, dateRest.split(' '))

            endYear, endMonth, dateRest2 = map(str, strDate2.split('-'))
            endDay, time = map(str, dateRest2.split(' '))

            startHr = '20'
            startMin ='00'

            endHr = '06'
            endMin = '30'


            URL = f'https://fclm-portal.amazon.com/reports/functionRollup?reportFormat=CSV&warehouseId=SIL2&processId=1003001&maxIntradayDays=1&spanType=Intraday&startDateIntraday={startYear}%2F{startMonth}%2F{startDay}&startHourIntraday={startHr}&startMinuteIntraday={startMin}&endDateIntraday={endYear}%2F{endMonth}%2F{endDay}&endHourIntraday={endHr}&endMinuteIntraday={endMin}'
            req = requests.Session()
            resp = req.get(URL, verify=False, allow_redirects=True, auth=HTTPKerberosAuth(mutual_authentication=OPTIONAL))
            data = io.StringIO(resp.text)
            df = pd.read_csv(data, usecols=['Employee Id', 'Paid Hours-Total(function,employee)', 'Jobs'])

            df.drop_duplicates(subset=['Employee Id'], keep='last', inplace=True)
            #help(df.drop)
            df = df.drop(columns=['Employee Id'])
            df = df.rename(columns = {'Paid Hours-Total(function,employee)': 'Total Hours'})

            #df.info()

            total = FHD_days[x]+" "+ "FHN "+ str(startMonth)+"/"+str(startDay)
            df.loc[total] = df.sum()
            df = df.astype({"Total Hours": float, "Jobs": int})
            df.round({'Total Hours': 2})
            totalRow=df.tail(1)
            df1 = pd.DataFrame(totalRow)
            #print(df1)

            df_empty = df_empty.append(df1)
        df_empty.to_csv("PPPicks_FHN.csv")
        print(df_empty)
    else:
        print('PPPicks for FHNs already collected! Continuing to next grab...')
funcRollPPPicksFHN()

PPPicks for FHNs already collected! Continuing to next grab...


In [456]:
def funcRollPPPicksBHN():
    if os.path.exists('PPPicks_BHN.csv')== False:
        df_empty = pd.DataFrame()
        for x in range(3):
            strDate = str(BHN_start[x])
            strDate2 = str(BHN_end[x])
            startYear, startMonth, dateRest = map(str, strDate.split('-'))
            startDay, time = map(str, dateRest.split(' '))

            endYear, endMonth, dateRest2 = map(str, strDate2.split('-'))
            endDay, time = map(str, dateRest2.split(' '))

            startHr = '20'
            startMin ='00'

            endHr = '06'
            endMin = '30'


            URL = f'https://fclm-portal.amazon.com/reports/functionRollup?reportFormat=CSV&warehouseId=SIL2&processId=1003001&maxIntradayDays=1&spanType=Intraday&startDateIntraday={startYear}%2F{startMonth}%2F{startDay}&startHourIntraday={startHr}&startMinuteIntraday={startMin}&endDateIntraday={endYear}%2F{endMonth}%2F{endDay}&endHourIntraday={endHr}&endMinuteIntraday={endMin}'
            req = requests.Session()
            resp = req.get(URL, verify=False, allow_redirects=True, auth=HTTPKerberosAuth(mutual_authentication=OPTIONAL))
            data = io.StringIO(resp.text)
            df = pd.read_csv(data, usecols=['Employee Id', 'Paid Hours-Total(function,employee)', 'Jobs'])

            df.drop_duplicates(subset=['Employee Id'], keep='last', inplace=True)
            #help(df.drop)
            df = df.drop(columns=['Employee Id'])
            df = df.rename(columns = {'Paid Hours-Total(function,employee)': 'Total Hours'})

            #df.info()

            total = BHD_days[x]+" "+ "BHN "+ str(startMonth)+"/"+str(startDay)
            df.loc[total] = df.sum()
            df = df.astype({"Total Hours": float, "Jobs": int})
            df.round({'Total Hours': 2})
            totalRow=df.tail(1)
            df1 = pd.DataFrame(totalRow)
            #print(df1)

            df_empty = df_empty.append(df1)
        df_empty.to_csv("PPPicks_BHN.csv")
        print(df_empty)
    else:
        print('PPPicks for BHNs already collected! Continuing to next grab...')
funcRollPPPicksBHN()

PPPicks for BHNs already collected! Continuing to next grab...


In [457]:
#Atlas Worksheet

#Used for pulling pack item missings and pick shorts
pickPackAtlas = r'https://atlas.corp.amazon.com/en_US/reporting?utf8=%E2%9C%93&report=1&aggregate_type=Warehouse+ID&user_id=&warehouse_id=SIL2&network_type=Fulfillment+Center&region_id=NA&manager_id=&workcell_type=&workcell_id=&fc_grouping=nttravi&query_type=Normal&shift_start_date=&shift_end_date=&shift_start_time=&shift_end_time=&start_time=2021-07-08+21%3A00&end_time=2021-07-09+06%3A30&target_process=outbound-department&pack_target_subprocess=none&sort_target_subprocess=none&problem_solve_target_subprocess=problem_solve_general_problem_solve&format=html'

#used for NME and Bin filter DPMO
StowQuality = r'https://atlas.corp.amazon.com/en_US/reporting?utf8=%E2%9C%93&report=1&aggregate_type=Warehouse+ID&user_id=&warehouse_id=SIL2&network_type=Fulfillment+Center&region_id=NA&manager_id=&workcell_type=&workcell_id=&fc_grouping=nttravi&query_type=Normal&shift_start_date=&shift_end_date=&shift_start_time=&shift_end_time=&start_time=2021-07-08+21%3A00&end_time=2021-07-09+06%3A30&target_process=stow-process&pack_target_subprocess=none&sort_target_subprocess=none&problem_solve_target_subprocess=problem_solve_general_problem_solve&format=html'

AtlasURL = [pickPackAtlas, StowQuality]


In [458]:
#Robo Scout Worksheet

#Used for No stow turn away and units per pod face

noStow = r'https://roboscout.amazon.com/analyze/19851/2826/?&sites=(SWA1)&datasource_current_day=false&datasource_startDateTime=2021-07-09%2002:00:00&datasource_endDateTime=2021-07-09%2011:30:00&mom_ids=1696%2C426&osm_ids=&oxm_ids=435&ofm_ids=&datasource_viz=nvd3Table'

#Used for Pick to Rebin only Pod gap %
RebinPodGap = r'https://roboscout.amazon.com/analyze/20990/?&sites=(SIL2)&datasource_current_day=false&datasource_startDateTime=2021-07-09%2011:35:00&datasource_endDateTime=2021-07-11%2001:00:00&mom_ids=1443&osm_ids=977&oxm_ids=1134&ofm_ids=598&datasource_viz=nvd3Table'

#Used for pick tak time
pickTacTime = r'https://roboscout.amazon.com/analyze/11355/?&sites=(SIL2)&datasource_current_day=false&datasource_startDateTime=2021-07-10%2002:00:00&datasource_endDateTime=2021-07-10%2011:30:00&mom_ids=26&osm_ids=608&oxm_ids=297&ofm_ids=769&datasource_viz=nvd3Table'

RoboScout = [noStow, RebinPodGap, pickTacTime]

In [459]:
#Dispatch Worksheet

#Used for pulling sort rates in Dispatch
dispatchSortRate = r'https://fclm-portal.amazon.com/ppa/inspect/process?primaryAttribute=CONTAINER_TYPE&secondaryAttribute=CONTAINER_TYPE&nodeType=DS&warehouseId=VIL2&processId=100021&startDateDay=2021%2F04%2F30&startDateWeek=2021%2F04%2F30&startDateMonth=2021%2F04%2F01&maxIntradayDays=1&spanType=Intraday&startDateIntraday=2021%2F07%2F09&startHourIntraday=21&startMinuteIntraday=0&endDateIntraday=2021%2F07%2F10&endHourIntraday=6&endMinuteIntraday=30'

#Used for induct rates PPA

inductRatePPA = r'https://fclm-portal.amazon.com/ppa/inspect/process?primaryAttribute=CONTAINER_TYPE&secondaryAttribute=CONTAINER_TYPE&nodeType=DS&warehouseId=VIL2&processId=100020&startDateDay=2021%2F04%2F30&startDateWeek=2021%2F04%2F30&startDateMonth=2021%2F04%2F01&maxIntradayDays=1&spanType=Intraday&startDateIntraday=2021%2F07%2F09&startHourIntraday=21&startMinuteIntraday=0&endDateIntraday=2021%2F07%2F10&endHourIntraday=6&endMinuteIntraday=30'

dispatchURL = [dispatchSortRate, inductRatePPA]

In [460]:
#TPH Worksheet

#Used for pulling dispatch volume
dispatchVolume = r'https://fclm-portal.amazon.com/ppa/inspect/node?nodeType=DS&warehouseId=VIL2&startDateDay=2021%2F04%2F30&startDateWeek=2021%2F04%2F30&startDateMonth=2021%2F04%2F01&maxIntradayDays=1&spanType=Intraday&startDateIntraday=2021%2F07%2F09&startHourIntraday=21&startMinuteIntraday=0&endDateIntraday=2021%2F07%2F10&endHourIntraday=6&endMinuteIntraday=30'

#Used for IB OB and total building TPH
inOutTPH = r'https://fclm-portal.amazon.com/reports/processPathRollup?reportFormat=HTML&warehouseId=SIL2&maxIntradayDays=1&spanType=Intraday&startDateIntraday=2021%2F07%2F09&startHourIntraday=21&startMinuteIntraday=0&endDateIntraday=2021%2F07%2F10&endHourIntraday=6&endMinuteIntraday=30&_adjustPlanHours=on&_hideEmptyLineItems=on&employmentType=AllEmployees'


TPH_URL = [dispatchVolume, inOutTPH]